In [1]:
!pip install --upgrade --quiet llama-index-core llama-index-llms-openai llama-index-tools-mcp llama-index-vector-stores-neo4jvector llama-index-embeddings-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


**LlamaIndex** is a data framework for building LLM-powered applications over your data. It provides data connectors for ingesting from diverse sources, powerful indexing and retrieval mechanisms, query engines and chat interfaces, and seamless integrations with vector stores, databases, and other LLM frameworks.

In this notebook, we'll build a company research agent that queries a Neo4j graph database.

In [2]:
import base64
import json

from llama_index.core.agent.workflow import (
    AgentStream,
    FunctionAgent,
    ToolCall,
    ToolCallResult,
)
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

LlamaIndex integrates with virtually every major LLM provider like OpenAI, Anthropic, Google, Cohere, Mistral, AWS Bedrock, Azure, and many more. This makes it easy to swap models or run comparisons without rewriting your application logic.

In this example, we'll use OpenAI as our LLM provider, specifically GPT-5.1

In [3]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key")

OpenAI API key··········


In [4]:
llm = OpenAI(model="gpt-5.1")

## Neo4j MCP Server

We'll start by using the official [Neo4j MCP Server](https://github.com/neo4j/mcp) to extend the agent with Neo4j tools. This MCP server provides the agent with capabilities to read the graph schema and execute Cypher queries, enabling it to fetch and analyze data directly from the database.

The following code installs the latest version on Google Colab and similar Linux-based systems. For other operating systems, please consult the [official installation documentation](https://neo4j.com/docs/mcp/current/installation/).

In [5]:
import requests

# Get latest release info from GitHub API
release = requests.get("https://api.github.com/repos/neo4j/mcp/releases/latest").json()
version = release["tag_name"]
print(f"Latest version: {version}")

# Download the latest Linux binary
!wget -q https://github.com/neo4j/mcp/releases/download/{version}/neo4j-mcp_Linux_x86_64.tar.gz

# Extract
!tar -xzf neo4j-mcp_Linux_x86_64.tar.gz

# Make executable
!chmod +x neo4j-mcp

# Cleanup
!rm neo4j-mcp_Linux_x86_64.tar.gz

# Move
!mv neo4j-mcp /usr/local/bin/

# Verify installation
!neo4j-mcp -v

Latest version: v1.1.1
neo4j-mcp version: v1.1.1


For this example, we'll use the companies database from the Neo4j demo server, which contains organizations, people, investors, and news articles.

For HTTP transport, you only need to set the `NEO4J_URI` and optionally `NEO4J_DATABASE` (if connecting to a specific database).

In [6]:
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com"
os.environ["NEO4J_DATABASE"] = "companies"
os.environ["NEO4J_MCP_TRANSPORT"] = "http"

# Run the server in the background
import subprocess
subprocess.Popen(["neo4j-mcp"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

<Popen: returncode: None args: ['neo4j-mcp']>

Here we create an MCP client that connects to our Neo4j server endpoint. Credentials are passed via the `Authorization` header using Basic authentication. The HTTP transport listens on port 80 by default, so the MCP endpoint is available at `http://localhost:80/mcp`. We then wrap it in a `McpToolSpec` to convert the available MCP tools into LlamaIndex-compatible tools. The to_tool_list_async() method discovers and returns all tools exposed by the server.

In [7]:
# Credentials are passed via bearer auth
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"

credentials = base64.b64encode(f"{os.environ["NEO4J_USERNAME"]}:{os.environ["NEO4J_PASSWORD"]}".encode()).decode()

# Connect to MCP server (SSE, HTTP, or local process)
mcp_client = BasicMCPClient(
    "http://localhost:80/mcp",
    headers={"Authorization": f"Basic {credentials}"},
)

mcp_tool_spec = McpToolSpec(
    client=mcp_client,
)

# Get tools (async recommended)
tools = await mcp_tool_spec.to_tool_list_async()


Here, we create a `FunctionAgent` with our Neo4j MCP tools.

In [8]:
workflow = FunctionAgent(
    tools=tools,
    llm=llm,
    system_prompt="You are an agent that can search in neo4j for relevant information. Use the get-schema before you execute any cypher queries",
)

Let's test it!

In [9]:
handler = workflow.run(user_msg="How many people are in the database?")

async for event in handler.stream_events():
    if isinstance(event, ToolCall):
        print(f"\n🔧 TOOL CALL: {event.tool_name}")
        print(f"   └─ Args: {event.tool_kwargs}")

    elif isinstance(event, ToolCallResult):
        print(f"   └─ Result: {event.tool_output}")
        print("-"*50)

    elif isinstance(event, AgentStream):
        if event.delta:
            print(event.delta, end="", flush=True)

response = await handler


🔧 TOOL CALL: get-schema
   └─ Args: {}
   └─ Result: meta=None content=[TextContent(type='text', text='[{"key":"_Bloom_Perspective_","value":{"type":"node","properties":{"data":"STRING","id":"STRING","name":"STRING","roles":"LIST","version":"STRING"},"relationships":{"_Bloom_HAS_SCENE_":{"direction":"out","labels":["_Bloom_Scene_"]}}}},{"key":"HAS_CEO","value":{"type":"relationship"}},{"key":"Organization","value":{"type":"node","properties":{"diffbotId":"STRING","id":"STRING","isDissolved":"BOOLEAN","isPublic":"BOOLEAN","motto":"STRING","name":"STRING","nbrEmployees":"INTEGER","revenue":"FLOAT","summary":"STRING"},"relationships":{"HAS_BOARD_MEMBER":{"direction":"out","labels":["Person"]},"HAS_CATEGORY":{"direction":"out","labels":["IndustryCategory"]},"HAS_CEO":{"direction":"out","labels":["Person"]},"HAS_COMPETITOR":{"direction":"out","labels":["Organization"]},"HAS_INVESTOR":{"direction":"out","labels":["Person","Organization"]},"HAS_SUBSIDIARY":{"direction":"out","labels":["Organ

## Custom Tools
Beyond using existing MCP servers, you can also implement your own custom tools and add them directly to the agent. This allows you to create specialized functionality tailored to your specific use case.

First, we need to define a Neo4j driver to establish a connection to the database and an `execute_cypher` helper method to run queries.



In [10]:
from neo4j import AsyncGraphDatabase

# Create async driver
driver = AsyncGraphDatabase.driver(
    os.environ["NEO4J_URI"],
    auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]),
)


async def execute_cypher(cypher: str, params: dict) -> list[dict]:
    result = await driver.execute_query(
        cypher, database_=os.environ["NEO4J_DATABASE"], **params
    )
    return [record.data() for record in result.records]


The first tool retrieves investors for a given company, returning their IDs, names, and types (e.g., Organization or Person).



In [11]:
async def get_investments(company: str) -> dict:
    """
    Returns the investments by a company by name.

    Args:
        company: The name of the company to look up investments for.

    Returns:
        List of investment ids, names and types.
    """
    try:
        results = await execute_cypher("""
            MATCH (o:Organization)-[:HAS_INVESTOR]->(i)
            WHERE o.name = $company
            RETURN i.id as id, i.name as name, head(labels(i)) as type
        """, {"company": company})
        return {"result": results}
    except Exception as e:
        return {"error": str(e)}

get_investments_tool = FunctionTool.from_defaults(
    async_fn=get_investments,
    name="get_investments",
    description="Returns the investments by a company by name. Returns list of investment ids, names and types."
)

Neo4j also provides native integrations with LlamaIndex like `Neo4jVectorStore` that can be wrapped as tools. Here we set up a vector store for semantic search over news articles, using a custom retrieval query to fetch article metadata alongside the matched chunks.

In [12]:
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool

neo4j_vector = Neo4jVectorStore(
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    url=os.environ["NEO4J_URI"],
    database=os.environ["NEO4J_DATABASE"],
    embedding_dimension=1536,
    index_name="news",
    node_label="Chunk",
    retrieval_query="""
        MATCH (node)<-[:HAS_CHUNK]-(a:Article)
        RETURN node.text AS text, score, node.id AS id,
        {date: a.date} AS metadata
    """,
)

index = VectorStoreIndex.from_vector_store(neo4j_vector)
query_engine = index.as_query_engine(similarity_top_k=5)

# As tool
news_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="news_search",
    description="Search for relevant news articles.",
)

Now we create a `FunctionAgent` that combines our MCP tools with the custom tools we defined above.

In [13]:
custom_workflow = FunctionAgent(
    tools=tools + [get_investments_tool, news_tool],
    llm=llm,
    system_prompt="You are an agent that can search in neo4j for relevant information. Use the get-schema before you execute any cypher queries",
)

Let's test it!

In [14]:
handler = custom_workflow.run(user_msg="Which companies did Google invest in?")

async for event in handler.stream_events():
    if isinstance(event, ToolCall):
        print(f"\n🔧 TOOL CALL: {event.tool_name}")
        print(f"   └─ Args: {event.tool_kwargs}")

    elif isinstance(event, ToolCallResult):
        print(f"   └─ Result: {event.tool_output}")
        print("-"*50)

    elif isinstance(event, AgentStream):
        if event.delta:
            print(event.delta, end="", flush=True)

response = await handler


🔧 TOOL CALL: get_investments
   └─ Args: {'company': 'Google'}
   └─ Result: {'result': [{'id': 'ELsv5bECSOiWG_Uhf_txI2w', 'name': 'Ionic Security', 'type': 'Organization'}, {'id': 'EUkm62r-bMOidNtPjTkdVvg', 'name': 'Avere Systems', 'type': 'Organization'}, {'id': 'EX-RLztfkOFqTLoM6xIVnlg', 'name': 'FlexiDAO', 'type': 'Organization'}, {'id': 'EtqXbQ9LaMGq8om4dhYY0Fw', 'name': 'Cloudflare', 'type': 'Organization'}, {'id': 'EWIvDLNCSMCCBYUyz0oFPVQ', 'name': 'Trifacta', 'type': 'Organization'}]}
--------------------------------------------------
Google has invested in the following companies (from the data available in this system):

1. Ionic Security  
2. Avere Systems  
3. FlexiDAO  
4. Cloudflare  
5. Trifacta  

If you want, I can provide a short description of each company and what they do.

In [15]:
handler = custom_workflow.run(user_msg="Any news abouy any acqusitions?")

async for event in handler.stream_events():
    if isinstance(event, ToolCall):
        print(f"\n🔧 TOOL CALL: {event.tool_name}")
        print(f"   └─ Args: {event.tool_kwargs}")

    elif isinstance(event, ToolCallResult):
        print(f"   └─ Result: {event.tool_output}")
        print("-"*50)

    elif isinstance(event, AgentStream):
        if event.delta:
            print(event.delta, end="", flush=True)

response = await handler


🔧 TOOL CALL: news_search
   └─ Args: {'input': 'company acquisitions M&A announced or completed'}
   └─ Result: Company acquisitions M&A announced or completed are a common strategic move in the business world today. Major companies like Accenture, Dentsu Aegis Network, Deloitte, and private equity firms have been actively involved in acquiring various firms to enhance their digital capabilities, expand their services, and tap into new markets. These acquisitions are driven by the need to adapt to changing technologies, increase domain expertise, and foster growth opportunities.
--------------------------------------------------
Here’s a quick snapshot of acquisition activity in general:

- Large consulting and tech-services firms (e.g., Accenture, Deloitte) continue to acquire smaller digital, cloud, and data/AI specialists to expand capabilities and geographic reach.
- Advertising and marketing groups (e.g., Dentsu, WPP, Publicis) regularly buy creative, digital, and performance-mar

## Summary
In this notebook, we built a company research agent using LlamaIndex with Neo4j:
1. **MCP Integration** — Connected to Neo4j using the official Neo4j MCP server for schema reading and Cypher queries
2. **FunctionAgent** — Created an agent workflow with `FunctionAgent` that orchestrates tool calls to answer complex questions
3. **Custom Tools** — Built specialized tools using `FunctionTool.from_defaults` with direct Neo4j driver queries and `Neo4jVectorStore` for semantic search

The LlamaIndex framework makes it straightforward to combine MCP servers with custom tools, swap LLM providers, and build composable agent workflows.